In [ ]:
import pandas as pd
import numpy as np
import networkx as nx
from scipy.stats import beta
import itertools

import matplotlib.pyplot as plt
import seaborn as sns

import plotly.graph_objects as go

sns.set_theme("paper", "white")

In [ ]:
df_users = pd.read_csv("./DSock/Active_Users.csv", index_col=0)
print(df_users.shape)
display(df_users.head())

In [ ]:
df_evals = pd.read_csv("./DSock/human_eval_analysis/attitude_tracking.csv")

df_evals = pd.merge(df_users, df_evals, how="left", left_on="MTurk_ID", right_on="id").rename({"id_x": "id"}, axis=1)

df_evals.index = [f"u{u}" for u in df_evals["id"]]

In [ ]:
topics = list("abcdefgh")
deltas = ["10", "21", "32", "43", "54"]

In [ ]:
timeline = pd.date_range("2020-09-03-03:59:59", periods=6, tz="utc")

get_change = lambda u, t: df_evals.loc[u][[f"{t}_delta_{d}" for d in deltas]].tolist()

In [ ]:
u_change = {
    u: {t: (np.array(get_change(u, t)) != 0).sum() for t in topics}
    for u in df_evals.index
}

## get events

+ post events -> nodes
+ comment events -> nodes
+ view events -> edges
+ like events -> edges
+ report events

In [ ]:
df_posts = pd.read_csv("./DSock/direct_influence/posts_with_senti_and_topic.csv", parse_dates=["createdAt"])
df_comts = pd.read_csv("./DSock/direct_influence/comments_with_senti_and_topic.csv", delimiter=",", parse_dates=["createdAt"]).dropna(subset=["CommenterId", "PostId"])
df_views = pd.read_csv("./DSock/postViews.csv", delimiter="|", parse_dates=["createdAt"]).dropna(subset=["UserId", "PostId"])
df_likes = pd.read_csv("./DSock/Likes.csv", delimiter="|", parse_dates=["createdAt"]).dropna(subset=["UserId", "PostId"])
df_repts = pd.read_csv("./DSock/ReportUsers.csv", delimiter="|", parse_dates=["createdAt"])

print(f"posts: {df_posts.shape}, comments: {df_comts.shape}, views: {df_views.shape}, likes: {df_likes.shape}, reports: {df_repts.shape}")

In [ ]:
df_likes["UserId"] = df_likes["UserId"].astype(int)
df_likes["PostId"] = df_likes["PostId"].astype(int)

df_comts["CommenterId"] = df_comts["CommenterId"].astype(int)
df_comts["PostId"] = df_comts["PostId"].astype(int)

In [ ]:
df_likes

### create auxiliary graph

In [ ]:
G = nx.DiGraph()
for u, uname, obsr, sock in df_users[["id", "username", "isObserver", "isPuppet"]].values:
    G.add_node(f"u{u}", id=u, kind="user", name=uname, observer=(obsr == "t"), sock=(sock == "t"))
    if obsr == "t":
        G.nodes[f"u{u}"]["color"] = "obsr"
    elif sock == "t":
        G.nodes[f"u{u}"]["color"] = "sock"
    else:
        G.nodes[f"u{u}"]["color"] = "part"

for p, author, t, senti, score, topic in df_posts[["PostID", "AuthorId", "createdAt", "labels", "scores", "topic_label"]].values:
    G.add_node(f"p{p}", id=p, kind="post", time=t, user=author, senti=senti, score=score, topic=topic)

for c, author, t, senti, score, topic in df_comts[["id", "CommenterId", "createdAt", "labels", "scores", "topic_label"]].values:
    G.add_node(f"c{c}", id=c, kind="comt", time=t, user=author, senti=senti, score=score, topic=topic)

for l, u, p, t in df_likes[["id", "UserId", "PostId", "createdAt"]].values:
    p_name = f"p{p}" if p in df_posts["PostID"] else f"c{p}"
    if f"u{u}" in G.nodes and p_name in G.nodes:
        G.add_node(f"l{l}", user=u, post=p_name, time=t, id=l, kind="like",
                   topic=G.nodes[p_name]["topic"], senti="like", score=1)

In [ ]:
# add views
for u, p, t, i in df_views[["UserId", "PostId", "createdAt", "id"]].values:
    p_name = f"p{p}" if p in df_posts["PostID"] else f"c{p}"
    if f"u{u}" in G.nodes and p_name in G.nodes:
        G.add_edge(f"u{u}", p_name, time=t, id=i, kind="view")

# add likes
# for u, p, t, i in df_likes[["UserId", "PostId", "createdAt", "id"]].values:
#     p_name = f"p{p}" if p in df_posts["PostID"] else f"c{p}"
#     if f"u{u}" in G.nodes and p_name in G.nodes:
#         G.add_edge(f"u{u}", p_name, time=t, id=i, kind="like")
for l, u, p, t in df_likes[["id", "UserId", "PostId", "createdAt"]].values:
    p_name = f"p{p}" if p in df_posts["PostID"] else f"c{p}"
    if f"u{u}" in G.nodes and p_name in G.nodes:
        G.add_edge(f"u{u}", f"l{l}", user=u, post=p_name, time=t, id=l, kind="like")
        G.add_edge(f"l{l}", p_name, user=u, post=p_name, time=t, id=l, kind="like")

# add create
for p, u, t in df_posts[["PostID", "AuthorId", "createdAt"]].values:
    if f"u{u}" in G.nodes:
        G.add_edge(f"p{p}", f"u{u}", time=t, kind="make")

for c, u, t, in df_comts[["id", "CommenterId", "createdAt"]].values:
    if f"u{u}" in G.nodes:
        G.add_edge(f"c{c}", f"u{u}", time=t, kind="make")

# for c, u, t in df_comts[["PostId", "CommenterId", "createdAt"]].values:
#     if f"u{u}" in G.nodes and f"c{c}" in G.nodes:
#         G.add_edge(f"c{c}", f"u{u}", kind="comt_by", time=t)

In [ ]:
def get_subgraph_with_node(G, node_name, topic="a"):
    # children = nx.descendants(G, source=node_name) | {node_name}
    tree = {
        e[1]: {
            "node": G.nodes[e[1]],
            "edge": G.edges[e],
            "time": G.edges[e]["time"],
        }
        # filter with topic
        for e in G.out_edges(node_name) if G.nodes[e[1]]["topic"] == topic
    }

    path = sorted(tree, key=lambda x: tree[x]["time"])
    
    return tree, path

In [ ]:
node_name = "u2337"
topic = "a"

tree, path = get_subgraph_with_node(G, node_name, topic)

print(len(path))

#### define colors and styles for graph drawing

In [ ]:
use_color = sns.color_palette("tab10")
use_color

In [ ]:
line_map = {
    "view": "-",
    "like": ".",
}

color_map = {
    "view": 4,
    "like": 1,
    
    "part": 2,
    "sock": 3,
    "obsr": 7,
    
    "make": 5,
    "comt": 9,
    "post": 0,
    
    "user": 8,
}

style_map = {
    "post": "o",
    "comt": "s",
}

In [ ]:
subG = nx.subgraph(G, path + [node_name])

pos = nx.spring_layout(subG, seed=3)
# pos = nx.drawing.nx_agraph.graphviz_layout(subG, prog="dot")
fig = plt.figure(figsize=(6, 6))
ax = fig.subplots()

nx.draw(
    subG, ax=ax,
    pos=pos,
    with_labels=True,
    node_color=[use_color[color_map[subG.nodes[n]["kind"]]]
                if n[0] != "u" else use_color[color_map[subG.nodes[n]["color"]]]
                for n in subG.nodes],
    edge_color=[use_color[color_map[subG.edges[e]["kind"]]] for e in subG.edges],
    labels={n: f"{n}\n{subG.nodes[n]['senti']}" if subG.nodes[n]['kind'] != "user" else n for n in subG.nodes},
    font_size=10,
)

nx.draw_networkx_edge_labels(
    subG, pos=pos, ax=ax,
    edge_labels={e: f"{subG.edges[e]['time']:%d-%H:%M}" for e in subG.edges}
)

# ax.set_title(f"Auxilary graph")
print("")

In [ ]:
user_timeline = {
    f"{node_name}_{i}": {
        "node": G.nodes[node_name],
        "time": timeline[i],
        "change": get_change(node_name, topic)[i-1] if i > 0 else 0,
    } for i in range(6)
}

print(user_timeline)

In [ ]:
tree_full = {**tree, **user_timeline}
path_full = sorted(path + list(user_timeline.keys()), key=lambda x: tree_full[x]["time"])

pathG = nx.DiGraph()

pathG.add_nodes_from(path_full)

for i in range(len(path_full)):
    for j in range(i+1, len(path_full)):
        x = path_full[i]
        y = path_full[j]
        if y[0] == "u" and x[0] != "u":
            pathG.add_edge(y, x, **G.edges[(y[:-2], x)])
            break

nx.set_node_attributes(
    pathG,
    {
        n: {
           "color": use_color[color_map[tree_full[n]["node"]["kind"]]],
        }
        for n in pathG.nodes
    }
)

nx.set_edge_attributes(
    pathG,
    {
        e: {"color": use_color[color_map[pathG.edges[e]["kind"]]]}
        for e in pathG.edges
    }
)

In [ ]:
pos = {}
for n in pathG.nodes:
    pos[n] = np.array([len(pos), 0])

fig = plt.figure(figsize=(14, 4))
ax = fig.subplots()

nx.draw(
    pathG,
    pos=pos,
    ax=ax,
    with_labels=True,
    font_size=10,
    node_color=[pathG.nodes[n]["color"] for n in pathG.nodes],
    edge_color=[pathG.edges[e]["color"] for e in pathG.edges],
    labels={n: f"{n}" if n[0]!="u" else f"{n}\n{tree_full[n]['change']}" for n in pathG.nodes},
    connectionstyle="arc3,rad=2",
)

## full influence graph

In [ ]:
node_name = "u2337"
topic = "a"

descend_full = [n for n in nx.descendants(G, node_name)
                if n[0] == "u" or G.nodes[n]["topic"] == topic]

np.random.seed(4)
# descend_full = np.random.choice(descend_full, 65, replace=False).tolist()

subG = G.subgraph(descend_full + [node_name]).copy()
print(len(subG))

subG = subG.subgraph([n for n in subG if nx.has_path(subG, node_name, n)])
print(len(subG))

In [ ]:
pos = nx.spring_layout(subG, iterations=6, seed=3)

fig = plt.figure(figsize=(8, 8))
ax = fig.subplots()

nx.draw(
    subG, ax=ax,
    pos=pos,
    with_labels=True,
    node_color=[use_color[color_map[subG.nodes[n]["kind"]]] for n in subG.nodes],
    edge_color=[use_color[color_map[subG.edges[e]["kind"]]] for e in subG.edges],
    labels={n: f"{n}\n{subG.nodes[n]['senti']}" if subG.nodes[n]['kind'] != "user" else n for n in subG.nodes},
    font_size=10,
)

# nx.draw_networkx_edge_labels(
#     subG, pos=pos, ax=ax,
#     edge_labels={e: f"{subG.edges[e]['time']:%d-%H:%M}" for e in subG.edges},
#     font_size=8,
# )

print(len(subG))

In [ ]:
# pos = nx.spring_layout(subG, iterations=100)
pos = nx.drawing.nx_agraph.graphviz_layout(subG, prog="dot")

fig = plt.figure(figsize=(14, 10))
ax = fig.subplots()

nx.draw(
    subG, ax=ax,
    pos=pos,
    with_labels=True,
#     node_color=[use_color[color_map[subG.nodes[n]["kind"]]] for n in subG.nodes],
    node_color=[use_color[color_map[subG.nodes[n]["kind"]]]
                if n[0] != "u" else use_color[color_map[subG.nodes[n]["color"]]]
                for n in subG.nodes],
    edge_color=[use_color[color_map[subG.edges[e]["kind"]]] for e in subG.edges],
    labels={n: f"{n}\n{subG.nodes[n]['senti']}" if subG.nodes[n]['kind'] != "user" else n for n in subG.nodes},
    font_size=10,
)

nx.draw_networkx_edge_labels(
    subG, pos=pos, ax=ax,
    edge_labels={e: f"{subG.edges[e]['time']:%d-%H:%M}" for e in subG.edges},
    font_size=8,
)

print(len(subG))

In [ ]:
attention_window = pd.Timedelta("1 day")
moment_time = timeline[5]
topic = "a"
pr_alpha = 0.85
beta_rv = beta(a=0.5, b=0.5)

node_name = "u2337"

print(f"{attention_window}, {moment_time}")

In [ ]:
moment_nodes = [n for n in G if n[0] == "u" or G.nodes[n]["topic"] == topic]

moment_graph = G.subgraph(moment_nodes).copy()
print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

# remove edges outside attention window
remove_edges = [e for e in moment_graph.edges if moment_graph.edges[e]["time"] > moment_time]
for n in moment_graph:
    if n[0] == "u" and not G.nodes[n]["observer"] and len(moment_graph.in_edges(n)) > 0:
        t_min = min([G.edges[e]["time"] for e in moment_graph.in_edges(n)])
        t_max = max([G.edges[e]["time"] for e in moment_graph.in_edges(n)])
        remove_edges += [e for e in moment_graph.out_edges(n) if not t_min - attention_window < moment_graph.edges[e]["time"] <= t_max]

print(f"remove edges {len(remove_edges)}")
moment_graph.remove_edges_from(remove_edges)
print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

# for source node node_name only
descend_full = [n for n in nx.descendants(moment_graph, node_name)]
moment_graph = moment_graph.subgraph(descend_full + [node_name]).copy()
print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")
moment_graph = moment_graph.subgraph([n for n in moment_graph if nx.has_path(moment_graph, node_name, n)]).copy()
print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

# remove isolated nodes
isolates = list(nx.isolates(moment_graph))
moment_graph.remove_nodes_from(isolates)
print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")


for node in moment_graph:
    elist = sorted(moment_graph.out_edges(node), key=lambda e: G.edges[e]["time"])
    x = np.linspace(0, 1, len(elist)+2)[1:-1]
    y = beta_rv.pdf(x)
    y = y / y.sum()
    for e, w in zip(elist, y):
        moment_graph.edges[e]["weight"] = w

pr_value = nx.pagerank_numpy(moment_graph, alpha=pr_alpha, weight="weight")

In [ ]:
# pos = nx.drawing.nx_agraph.graphviz_layout(moment_graph, prog="dot")
pos = nx.drawing.nx_agraph.graphviz_layout(moment_graph)
# pos = nx.spring_layout(moment_graph, iterations=3, seed=3)

fig = plt.figure(figsize=(14, 10))
ax = fig.subplots()

nx.draw(
    moment_graph, ax=ax,
    pos=pos,
    with_labels=True,
#     node_color=[use_color[color_map[moment_graph.nodes[n]["kind"]]] for n in moment_graph.nodes],
    node_color=[use_color[color_map[moment_graph.nodes[n]["kind"]]]
                if n[0] != "u" else use_color[color_map[moment_graph.nodes[n]["color"]]]
                for n in moment_graph.nodes],
    edge_color=[use_color[color_map[moment_graph.edges[e]["kind"]]] for e in moment_graph.edges],
    labels={n: f"{n}\n{moment_graph.nodes[n]['senti']}" if moment_graph.nodes[n]['kind'] != "user" else n for n in moment_graph.nodes},
    font_size=10,
    node_size=[pr_value[n]*len(moment_graph)*100 for n in moment_graph],
)

nx.draw_networkx_edge_labels(
    subG, pos=pos, ax=ax,
#     edge_labels={e: f"{moment_graph.edges[e]['time']:%d-%H:%M}" for e in moment_graph.edges},
    edge_labels={e: "" for e in moment_graph.edges},
    font_size=8,
)

print(len(moment_graph))

In [ ]:
def compute_pagerank(G, node_name, pr_alpha, beta_a, beta_b, moment_time, attention_window, topic=None):
    if topic == None:
        moment_nodes = G.nodes
    else:
        moment_nodes = [n for n in G if n[0] == "u" or G.nodes[n]["topic"] == topic]

    moment_graph = G.subgraph(moment_nodes).copy()
    print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

    # remove edges outside attention window
    remove_edges = [e for e in moment_graph.edges if moment_graph.edges[e]["time"] > moment_time]
    for n in moment_graph:
        if n[0] == "u" and not G.nodes[n]["observer"] and len(moment_graph.in_edges(n)) > 0:
            t_min = min([G.edges[e]["time"] for e in moment_graph.in_edges(n)])
            t_max = max([G.edges[e]["time"] for e in moment_graph.in_edges(n)])
            remove_edges += [e for e in moment_graph.out_edges(n) if not t_min - attention_window < moment_graph.edges[e]["time"] <= t_max]

#     print(f"remove edges {len(remove_edges)}")
    moment_graph.remove_edges_from(remove_edges)
#     print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

    # for source node node_name only
    descend_full = [n for n in nx.descendants(moment_graph, node_name)]
    moment_graph = moment_graph.subgraph(descend_full + [node_name]).copy()
#     print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")
    moment_graph = moment_graph.subgraph([n for n in moment_graph if nx.has_path(moment_graph, node_name, n)]).copy()
#     print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")

    # remove isolated nodes
    isolates = list(nx.isolates(moment_graph))
    moment_graph.remove_nodes_from(isolates)
    print(f"nodes {len(moment_graph)}, edges: {len(moment_graph.edges)}")


    for node in moment_graph:
        elist = sorted(moment_graph.out_edges(node), key=lambda e: G.edges[e]["time"])
        x = np.linspace(0, 1, len(elist)+2)[1:-1]
        y = beta_rv.pdf(x)
        y = y / y.sum()
        for e, w in zip(elist, y):
            moment_graph.edges[e]["weight"] = w

    pr_value = nx.pagerank_numpy(moment_graph, alpha=pr_alpha, weight="weight")
    return pr_value, moment_graph

In [ ]:
attention_window = pd.Timedelta("1 day")
moment_time = timeline[5]
topic = "a"
pr_alpha = 0.85
beta_rv = beta(a=0.5, b=0.5)

node_name = "u2337"

pr_value, moment_graph = compute_pagerank(G, node_name=node_name, pr_alpha=pr_alpha, beta_a=0.5, beta_b=0.5, moment_time=moment_time, attention_window=attention_window, topic=None)

In [ ]:
obsr_list = [f"u{u}" for u in df_users[df_users["isObserver"] == "t"]["id"]]

In [ ]:
from tqdm.notebook import tqdm
# from multiprocessing import Pool
from joblib import Parallel, delayed
from pathlib import Path

In [ ]:
keyq = [(node_name, t, pr_alpha, beta_a, beta_b)
    for node_name in obsr_list for t in timeline[1:]
    for pr_alpha, beta_a, beta_b in itertools.product(np.linspace(0.1, 0.9, 5), np.linspace(0.1, 0.9, 5), np.linspace(0.1, 0.9, 5))
       ]

print(f"jobs: {len(keyq)}")

In [ ]:
def short_compute_pagerank(n, p, a, b, t):
    return compute_pagerank(
    G, node_name=n, pr_alpha=p, beta_a=a, beta_b=b, moment_time=t, attention_window=attention_window, topic=None)

pr_dict_fp = Path("res/pr_dict.pkl")

if pr_dict_fp.exists():
    print("load from file")
    with open(pr_dict_fp, "rb") as fp:
        pr_dict = pickle.load(fp)
else:
    print("generate and save to file")
    valueq = Parallel(n_jobs=30)(delayed(short_compute_pagerank)(*tup) for tup in tqdm(keyq))

# valueq = pool.starmap(
#     func=short_compute_pagerank,
#     iterable=tqdm(keyq),
#     chunksize=1,
# )

### big number of values to compute
In total, there will be **21,250** jobs

+ Number of survey time for each user: 5
+ Number of observers: 34
+ Number of parameters
    - Pagerank damping factor: 5
    - Beta alpha, beta: 5*5=25

In [ ]:
import pickle

In [ ]:
with open("res/pr_dict.pkl", "wb") as fp:
    dictq = dict(zip(keyq, [v[0] for v in valueq]))
    pickle.dump(dictq, fp)